# Load complex_system.xml and make a `Simulation` obj

We will ignore equilibration, which is something you probably don't want to do in production.

In [2]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *

import numpy as np

from sys import stdout
import sys
from tqdm import tqdm



In [3]:
#load complex
complex_pdb = PDBFile('./openmm_simulation/complex_coords.pdb')
complex_system = XmlSerializer.deserialize(open('./openmm_simulation/complex_system_.xml').read())



In [4]:
##add a barostat. Not only will this initially fix any vacuum between periodic cells,
##it will be maintained throughout to keep pressure reasonable. 
barostat = MonteCarloBarostat(1*bar, 310*kelvin)
complex_system.addForce(barostat)

5

In [6]:
#OpenMM simulation machinery:
integrator = LangevinIntegrator(310*kelvin, 10/picosecond, 0.002*picoseconds)
platform = Platform.getPlatformByName('OpenCL')
prop = {'OpenCLPrecision':'single', 'OpenCLDeviceIndex':'0'}
simulation = Simulation(complex_pdb.topology, complex_system, integrator, platform, prop)


In [7]:
#minimize starting coords:
simulation.context.setPositions(complex_pdb.positions)
simulation.minimizeEnergy(maxIterations=150)

# Add a DCDReporter, and simulate
takes about three hours 

In [ ]:
simulation.reporters.append(DCDReporter('./traj_production.dcd', 10000)) #20ps per sample
simulation.reporters.append(StateDataReporter(stdout, 10000, 
                                              step=True, 
                                              potentialEnergy=True, 
                                              temperature=True, speed=True))


##Extra 1 ns for good measure.
step_per_ns = 500000
num_ns = 50
simulation.step(step_per_ns * num_ns)


#"Step","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
10000,-244078.046935851,314.1161081244298,0
20000,-244802.8079700591,306.94958800815084,328
30000,-244209.9312187899,309.86895505168525,329
40000,-245816.08737655776,313.1231928279203,329
50000,-245414.61540736235,314.0271526116457,327
60000,-245148.30666596093,310.83749495522375,327
70000,-244711.9812098709,309.3263354980667,326
80000,-243991.23943777964,310.329007137759,325
